In [58]:
#!pip install folium

In [59]:
import pandas as pd
import folium
from geopy.distance import vincenty as get_geodesic_distance

In [60]:
df_os = pd.read_csv("https://raw.githubusercontent.com/fabioseidl/hackathon_unifique/main/dados/ordens_servico.csv")
tecnico = {"nome":"Fabio Seidl", "latitude":-27.594956700750053, "longitude":-48.535944492550385}
display(df_os)

,id_os,data,tecnico,descricao_os,latitude,longitude,status
0,1,16/05/2021,Fabio Seidl,NOVA INSTALAÇÃO FIBRA,-27.594002,-48.551428,PREVISTO
1,2,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.590036,-48.550490,PREVISTO
2,3,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.595641,-48.504101,PREVISTO
3,4,16/05/2021,Fabio Seidl,CONEXÃO INTERMITENTE,-27.585714,-48.543712,PREVISTO
4,5,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.575976,-48.508779,PREVISTO


In [61]:
map = folium.Map(location=[-27.594956700750053, -48.535944492550385], zoom_start=14, width=900, height=600)

for os in df_os.iterrows():
    folium.Marker(
        [os[1].latitude, os[1].longitude], popup=str(os[1].id_os)
    ).add_to(map)

    folium.PolyLine([[tecnico["latitude"], tecnico["longitude"]], [os[1].latitude, os[1].longitude]], weight=5, color="orange").add_to(map)

folium.Marker(
        [tecnico["latitude"], tecnico["longitude"]], popup=tecnico["nome"],
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map)

map

In [62]:
def get_distance(row):
    source_lonlat = row.longitude_origem, row.latitude_origem
    target_lonlat = row.longitude, row.latitude
    return int(get_geodesic_distance(target_lonlat, source_lonlat).meters)

In [63]:
df_os = df_os.append({"id_os":6, "data":"16/05/2021", "tecnico":"Fabio Seidl", "descricao_os":"OS EXTRA", "latitude":-27.611343047576028, "longitude":-48.545239153947065, "status":"PENDENTE"}, ignore_index=True).reset_index()

In [64]:
map = folium.Map(location=[-27.594956700750053, -48.535944492550385], zoom_start=14, width=900, height=600)

df_intervals = df_os.copy()
df_final = pd.DataFrame()
anterior = {}

for idx, (_, os) in enumerate(df_os.iterrows()):

    if(idx <= 0):
        df_intervals["latitude_origem"] = tecnico["latitude"]
        df_intervals["longitude_origem"] = tecnico["longitude"]
        df_intervals["anteior_id"] = 0
    else:
        df_intervals["latitude_origem"] = anterior["latitude"]
        df_intervals["longitude_origem"] = anterior["longitude"]
        df_intervals["anteior_id"] = anterior["id_os"]
        #print(anterior["id_os"])

    df_intervals["distancia"] = df_intervals.apply(get_distance, axis=1)
    df_intervals = df_intervals.sort_values("distancia")

    first_row = df_intervals[:1]
    df_final = df_final.append(first_row)

    anterior["latitude"] = first_row.iloc[0]["latitude"]
    anterior["longitude"] = first_row.iloc[0]["longitude"]
    anterior["id_os"]   = first_row.iloc[0]["id_os"]
    
    df_intervals = df_intervals.iloc[1:]

for row in df_final.iterrows():
    folium.Marker(
        [row[1].latitude, row[1].longitude], popup=row[1].descricao_os
    ).add_to(map)

    folium.PolyLine([[row[1].latitude_origem, row[1].longitude_origem], [row[1].latitude, row[1].longitude]], weight=5, color="green").add_to(map)

folium.Marker(
        [tecnico["latitude"], tecnico["longitude"]], popup=tecnico["nome"],
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map)

map

In [65]:
display(df_final)

,index,id_os,data,tecnico,descricao_os,latitude,longitude,status,latitude_origem,longitude_origem,anteior_id,distancia
3,3,4,16/05/2021,Fabio Seidl,CONEXÃO INTERMITENTE,-27.585714,-48.543712,PREVISTO,-27.594957,-48.535944,0,1100
1,1,2,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.590036,-48.550490,PREVISTO,-27.585714,-48.543712,4,818
0,0,1,16/05/2021,Fabio Seidl,NOVA INSTALAÇÃO FIBRA,-27.594002,-48.551428,PREVISTO,-27.590036,-48.550490,2,310
5,5,6,16/05/2021,Fabio Seidl,OS EXTRA,-27.611343,-48.545239,PENDENTE,-27.594002,-48.551428,1,1453
2,2,3,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.595641,-48.504101,PREVISTO,-27.611343,-48.545239,6,4719
4,4,5,16/05/2021,Fabio Seidl,SEM CONEXÃO,-27.575976,-48.508779,PREVISTO,-27.595641,-48.504101,3,1543
